In [10]:
import pandas as pd
import time
from datetime import datetime
from fbprophet import Prophet

In [59]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
bucket = "04abb7992a07e000"
client = InfluxDBClient(url="http://localhost:9999", token="SVwyqngOyVUM7Ar0DB6nU2rA0il010F1r7wck1a24XF0JDfq15W1Xgz7FTAN3IQk8PkQH9KRoVkVg4hSxT1fIQ==", org="Org")
query_api = client.query_api()
write_api = client.write_api(write_options=SYNCHRONOUS)

In [117]:
query = 'from(bucket:"pandas ")' \
        ' |> range(start: -3h)'\
        ' |> filter(fn: (r) => r._measurement == "activity" or r._measurement == "weather")' \
        ' |> filter(fn: (r) => r.location == "eur" or r.location == "us")'
result = client.query_api().query(org="Org", query=query)



In [204]:
raw = []
record = []
for table in result:
    for record in table.records:
        raw.append((record.get_field(), record.get_value(), record.get_time()))
#         print(record.values["_time"], record.values["_field"]) 
        print(record.values.get("_value"), record.values.get("type"))


6.0 ballet
8.0 tennis
9.0 soccer
81.0 None
70.0 None
50.0 None
1.0 tag
2.0 swimming
70.0 None
50.0 None
51.0 None
5.0 tennis
6.0 soccer
2.0 tag
51.0 None
7.0 football
1.0 swimming
3.0 football
81.0 None
5.0 ballet


In [62]:
raw[0:5]

[('fun_level',
  6.0,
  datetime.datetime(2019, 10, 24, 18, 10, 17, 529757, tzinfo=datetime.timezone.utc)),
 ('fun_level',
  8.0,
  datetime.datetime(2019, 10, 24, 18, 10, 17, 529757, tzinfo=datetime.timezone.utc)),
 ('fun_level',
  9.0,
  datetime.datetime(2019, 10, 24, 18, 10, 17, 529757, tzinfo=datetime.timezone.utc)),
 ('temperature',
  81.0,
  datetime.datetime(2019, 10, 24, 18, 10, 17, 529757, tzinfo=datetime.timezone.utc)),
 ('humidity',
  70.0,
  datetime.datetime(2019, 10, 24, 18, 10, 17, 529757, tzinfo=datetime.timezone.utc))]

In [66]:
df=pd.DataFrame(raw, columns=['field','value','datetime'], index=None)
df['datetime'] = df['datetime'].values.astype('<M8[D]')
df.head()

,field,value,datetime
0,fun_level,6.0,2019-10-24
1,fun_level,8.0,2019-10-24
2,fun_level,9.0,2019-10-24
3,temperature,81.0,2019-10-24
4,humidity,70.0,2019-10-24


In [202]:
def build_flux_query(bucket,measurement,start,stop):
    from_range = 'from(bucket:"' + bucket + '")' \
                 ' |> range(start:' + start + ', stop:' + stop +')'
    if len(measurement) == 1:
        measurement = ' |> filter(fn: (r) => r._measurement == "' + measurement[0] 
    else:
        m = []
        for i in range(1,len(measurement)):
            m.append(' or r._measurement == "' + measurement[i] + '"') 
        measurement = ' |> filter(fn: (r) => r._measurement == "' + measurement[0] + '"' + "".join(m) + ')' 
    query = from_range + measurement 
    return query
   

In [203]:
build_flux_query("pandas",["activity","weather","test"],"-5h","-1h")

'from(bucket:"pandas") |> range(start:-5h, stop:-1h) |> filter(fn: (r) => r._measurement == "activity" or r._measurement == "weather" or r._measurement == "test")'

In [166]:
str(y)

'[\'or r._measurement == "today"\', \'or r._measurement == "yest"\', \'or r._measurement == "yew"\']'

In [76]:

def get_dataframe(precision,query,org,):
    result = client.query_api().query(org="Org", query=query)
    raw = []
    for table in result:
        for record in table.records:
            raw.append((record.get_field(), record.get_value(), record.get_time()))
    df=pd.DataFrame(raw, columns=['field','value','datetime'], index=None)
    df['datetime'] = df['datetime'].values.astype('<M8[' + precision + ']')
    return df


In [80]:
get_dataframe('ns',query,"Org")

,field,value,datetime
0,fun_level,6.0,2019-10-24 18:10:17.529757
1,fun_level,8.0,2019-10-24 18:10:17.529757
2,fun_level,9.0,2019-10-24 18:10:17.529757
3,temperature,81.0,2019-10-24 18:10:17.529757
4,humidity,70.0,2019-10-24 18:10:17.529757
5,humidity,50.0,2019-10-24 18:10:17.529757
6,fun,1.0,2019-10-24 17:51:22.076670
7,fun,2.0,2019-10-24 17:51:22.076670
8,humidity,70.0,2019-10-24 18:10:17.529757
9,humidity,50.0,2019-10-24 18:10:17.529757


In [118]:
class FluxStructure:
    pass


class FluxTable(FluxStructure):

    def __init__(self) -> None:
        self.columns = []
        self.records = []

    def get_group_key(self):
        return list(filter(lambda column: (column.group is True), self.columns))

    def __str__(self):
        cls_name = type(self).__name__
        return cls_name + "() columns: " + str(len(self.columns)) + ", records: " + str(len(self.records))

    def __iter__(self):
        return iter(self.records)


class FluxColumn(FluxStructure):
    def __init__(self, index=None, label=None, data_type=None, group=None, default_value=None) -> None:
        self.default_value = default_value
        self.group = group
        self.data_type = data_type
        self.label = label
        self.index = index


class FluxRecord(FluxStructure):

    def __init__(self, table, values=None) -> None:
        if values is None:
            values = {}
        self.table = table
        self.values = values

    def get_start(self):
        return self.values.get("_start")

    def get_stop(self):
        return self.values.get("_stop")

    def get_time(self):
        return self.values.get("_time")

    def get_value(self):
        return self.values.get("_value")

    def get_field(self):
        return self.values.get("_field")

    def get_measurement(self):
        return self.values.get("_measurement")
    
    def get_type(self):
        return self.values.get("type")

    def __str__(self):
        cls_name = type(self).__name__
        return cls_name + "() table: " + str(self.table) + ", " + str(self.values)
    
raw = []
record = []
for table in result:
    for record in table.records:
        raw.append((record.get_field(), record.get_value(), record.get_time()), record.get_type())
#         print(record.values["_time"], record.values["_field"]) 
        print(record)

AttributeError: 'FluxRecord' object has no attribute 'get_type'